# Lecture Report 2: Analysis of Tullock's (1980) "Efficient Rent Seeking"

**Student:** Ondřej Marvan
**Date:** 17 November 2025

This notebook performs a CAS (Computer Algebra System) analysis of the production function used in Gordon Tullock's (1980) seminal paper on "rent-seeking."

**Article:** Tullock, G. (1980). "Efficient Rent Seeking." In J. M. Buchanan, R. D. Tollison, & G. Tullock (Eds.), *Toward a Theory of the Rent-Seeking Society* (pp. 97-112). Texas A&M University Press.

**Goals:**
1.  **Specify** the "Contest Success Function" (CSF) in SymPy.
2.  **Test** its core mathematical properties (monotonicity, concavity, returns to scale).
3.  **Verify Internal Validity** by symbolically solving for the Nash Equilibrium.
4.  **Assess External Validity** by analyzing the model's predictions.

## Step 2: CAS Specification

In [1]:
import sympy as sp
from sympy import symbols, Function, diff, simplify, solve

# Enable pretty printing for nice mathematical output
sp.init_printing(use_unicode=True)

# 1. DEFINE SYMBOLIC VARIABLES
# We set assumptions (positive=True) because spending (x), the prize (V),
# and the decisiveness parameter (r) must be positive.
x_A, x_B, V, r = symbols('x_A x_B V r', positive=True, real=True)

print(f"Defined inputs:")
print(f"  Player A's spending: {x_A}")
print(f"  Player B's spending: {x_B}")
print(f"  Value of the prize: {V}")
print(f"  Decisiveness parameter: {r}")


# 2. DEFINE THE 'PRODUCTION FUNCTION' (TULLOCK CSF)
# This is p_A = (x_A^r) / (x_A^r + x_B^r)
# It 'produces' the probability of Player A winning.
p_A = (x_A**r) / (x_A**r + x_B**r)

print("\nDefined 'Production Function' (Tullock CSF):")
sp.pprint(p_A)


# 3. DEFINE THE UTILITY FUNCTION FOR PLAYER A
# U_A = (Probability of Winning * Value of Prize) - Cost of Spending
U_A = (p_A * V) - x_A

print("\nDefined Player A's Utility Function (U_A):")
sp.pprint(U_A)

Defined inputs:
  Player A's spending: x_A
  Player B's spending: x_B
  Value of the prize: V
  Decisiveness parameter: r

Defined 'Production Function' (Tullock CSF):
      r    
   x_A     
───────────
   r      r
x_A  + x_B 

Defined Player A's Utility Function (U_A):
       r         
  V⋅x_A          
─────────── - x_A
   r      r      
x_A  + x_B       


## Step 3: Property Tests

Now I will test the core mathematical properties of the "production function" `p_A` to ensure it is well-behaved.

### Test 1: Monotonicity
Does spending more money *always* increase the probability of winning? We check the sign of the first partial derivative ($\partial p_A / \partial x_A$). It should be positive.

In [2]:
# Test 1: Monotonicity
dpA_dxA = diff(p_A, x_A)
print("First partial derivative (dp_A / dx_A):")
sp.pprint(simplify(dpA_dxA))

# All variables (r, x_A, x_B) are positive, so the numerator and denominator
# are both positive. The derivative is POSITIVE.
# Test PASSED.

First partial derivative (dp_A / dx_A):
            r - 1    r       
       r⋅x_A     ⋅x_B        
─────────────────────────────
   2⋅r        r    r      2⋅r
x_A    + 2⋅x_A ⋅x_B  + x_B   


### Test 2: Concavity (Diminishing Returns)
Is the *second* million dollars spent less effective than the *first*? This implies diminishing returns. We check the sign of the second partial derivative ($\partial^2 p_A / \partial x_A^2$). It should be negative.

The general symbolic form is complex. To check the property, I'll substitute the standard case from the literature where `r=1` (a simple lottery).

### Test 2: Concavity (Diminishing Returns)
Is the *second* million dollars spent less effective than the *first*? This implies diminishing returns. We check the sign of the second partial derivative ($\partial^2 p_A / \partial x_A^2$). It should be negative.

The general symbolic form is complex. To check the property, I'll substitute the standard case from the literature where `r=1` (a simple lottery).

In [3]:
# Test 2: Concavity
# The full second derivative is complex to interpret by eye.
d2pA_dxA2_full = diff(p_A, x_A, 2)

# Let's substitute r=1 to check the sign.
d2pA_dxA2_r1 = d2pA_dxA2_full.subs(r, 1)
print("Second derivative when r=1:")
sp.pprint(simplify(d2pA_dxA2_r1))

# The result is -2*x_B / (x_A + x_B)**3.
# Since x_A and x_B are positive, this expression is always NEGATIVE.
# Test PASSED. The function exhibits diminishing returns.

Second derivative when r=1:
  -2⋅x_B    
────────────
           3
(x_A + x_B) 


### Test 3: Returns to Scale (Homogeneity)
What happens if *both* players double their spending (or scale by `t`)? We test `p_A(t*x_A, t*x_B)` to check the function's homogeneity.

In [4]:
# Test 3: Returns to Scale
t = symbols('t', positive=True, real=True)
p_A_scaled = p_A.subs([(x_A, t*x_A), (x_B, t*x_B)])
print("Testing p_A(t*x_A, t*x_B):")
sp.pprint(simplify(p_A_scaled))

# The scaling factor 't' completely cancels out.
# This means p_A is homogeneous of degree 0.
#
# Economic Insight: If both players double their spending,
# their probabilities of winning DO NOT CHANGE. This is the
# mathematical core of the "social waste" argument.

Testing p_A(t*x_A, t*x_B):
      r    
   x_A     
───────────
   r      r
x_A  + x_B 


## Step 4: Internal Validity - Solving for Nash Equilibrium

Now, I'll test Tullock's central claim: that there is a predictable, wasteful equilibrium level of spending.
To find this, we find the First-Order Condition (FOC) for Player A by taking the derivative of their utility `U_A` and setting it to 0. This is the point where the marginal benefit of spending equals its marginal cost (which is 1).

In [5]:
# 1. Find the First-Order Condition (FOC)
FOC_A = diff(U_A, x_A)
print("First-Order Condition (dU_A / dx_A):")
sp.pprint(simplify(FOC_A))

First-Order Condition (dU_A / dx_A):
         r    r       
  V⋅r⋅x_A ⋅x_B        
────────────────── - 1
                 2    
    ⎛   r      r⎞     
x_A⋅⎝x_A  + x_B ⎠     


### Solving for Symmetric Equilibrium
The FOC is complex. To solve it, we can assume a *symmetric* equilibrium. In this case, both rational players will choose to spend the same amount.

We set `x_A = x_B = x*` (which we'll call `x_star`).

In [6]:
# 2. Substitute for symmetric equilibrium
x_star = symbols('x_star', positive=True, real=True)
FOC_symmetric = FOC_A.subs([(x_A, x_star), (x_B, x_star)])

print(f"Symmetric FOC (setting x_A=x_B={x_star}):")
sp.pprint(simplify(FOC_symmetric))

# The complex FOC simplifies beautifully to (V*r) / (4*x_star) - 1

Symmetric FOC (setting x_A=x_B=x_star):
V⋅r        
─── - xₛₜₐᵣ
 4         
───────────
   xₛₜₐᵣ   


### Finding the Equilibrium Spending and Total Waste
Now we ask SymPy to solve this simplified equation `(V*r) / (4*x_star) - 1 = 0` for `x_star`. This will give us the Nash Equilibrium spending for *one* player.

We will then calculate the **Total Social Waste** (the amount spent by both players).

In [7]:
# 3. Ask CAS to solve for x_star
# We set the FOC to 0 and solve for x_star
equilibrium_solutions = solve(FOC_symmetric, x_star)
equilibrium_spending = equilibrium_solutions[0]

print("--- CAS Solution for Equilibrium Spending (x*) ---")
sp.pprint(equilibrium_spending)


# 4. Calculate Total Social Waste
# This is the sum of spending from both players (x* + x*)
total_waste = 2 * equilibrium_spending

print("\n--- Total Social Waste (x_A* + x_B*) ---")
sp.pprint(total_waste)

# This symbolically proves Tullock's central claim.
# Internal Validity: PASSED.

--- CAS Solution for Equilibrium Spending (x*) ---
V⋅r
───
 4 

--- Total Social Waste (x_A* + x_B*) ---
V⋅r
───
 2 


## Step 5: External Validity - The "Tullock Paradox"

The internal validity holds. The model is mathematically consistent.
Now, let's check the external validity by plugging plausible values into our derived formula:
`Total Waste = (V * r) / 2`

The "Tullock Paradox" is the real-world observation that lobbying spending (Total Waste) often appears *much lower* than the value of the prize (V).

Let's test our formula with different values for `r` (the decisiveness of the contest).

In [8]:
# 1. Define the waste formula from our CAS result
waste_formula = total_waste

# Case 1: Standard lottery (r=1)
# This is the base case in many papers.
waste_r1 = waste_formula.subs(r, 1)
print("Total waste when r=1 (lottery):")
sp.pprint(waste_r1)
print(f"Interpretation: Players waste {100 * (waste_r1/V)}% of the prize value.\n")


# Case 2: More decisive contest (r=2)
# Here, a small advantage in spending gives a big advantage in winning.
waste_r2 = waste_formula.subs(r, 2)
print("Total waste when r=2 (decisive contest):")
sp.pprint(waste_r2)
print(f"Interpretation: Players waste {100 * (waste_r2/V)}% of the prize value! (Full dissipation)\n")


# Case 3: Less decisive contest (r=0.5)
# Here, spending is very ineffective (strong diminishing returns).
waste_r0_5 = waste_formula.subs(r, 0.5)
print("Total waste when r=0.5 (less decisive):")
sp.pprint(waste_r0_5)
print(f"Interpretation: Players waste {100 * (waste_r0_5/V)}% of the prize value.\n")

Total waste when r=1 (lottery):
V
─
2
Interpretation: Players waste 50% of the prize value.

Total waste when r=2 (decisive contest):
V
Interpretation: Players waste 100% of the prize value! (Full dissipation)

Total waste when r=0.5 (less decisive):
0.25⋅V
Interpretation: Players waste 25.0000000000000% of the prize value.



## Step 6: Conclusions from CAS Analysis

Our CAS analysis is complete and has yielded several key insights:

1.  **Properties (Internal):** The "production function" for winning (`p_A`) is well-behaved: it's monotonic (more spending helps) and concave (it has diminishing returns).
2.  **Social Waste (Internal):** The function is homogeneous of degree 0, which mathematically proves the "social waste" argument: if all players increase spending, no one is better off.
3.  **Equilibrium (Internal):** The model is **internally valid**. We symbolically derived the Nash Equilibrium for total spending: **Total Waste = (V * r) / 2**.
4.  **Tullock Paradox (External):** The external validity of the model depends *entirely* on the `r` parameter.
    * The paradox (that real-world waste seems low) is **resolved** if real-world contests are "indecisive" (i.e., **r < 1**). If `r=0.5`, only 25% of the prize value is wasted, which aligns much better with empirical observations.
    * If `r = 2`, the model predicts *full dissipation* (100% of the prize is wasted), which is empirically unlikely.
    * If `r > 2`, the model predicts players would waste *more* than the prize value, which is nonsensical.
    * Therefore, the model is only externally plausible when `r` is assumed to be in the range (0, 2].